node scripts/export_analysis3_csv.js --start 2025-12-01 --end 2025-12-31を実行する

## 分析3：今日のプラン遵守度の分析

本分析では，SPI や EAC に基づいて生成される
「今日のプラン」が，
利用者の実際の行動選択に
どの程度反映されているかを検証する。

提示されたプランが，
単なる情報提示にとどまらず，
行動選択や時間配分に影響を与えているかを確認することが目的である。


## データの読み込み（分析3：今日のプラン遵守度）

本分析では，「今日のプラン」が実際の行動にどの程度反映されているかを検証するため，
日次単位で出力した 2 種類の CSV を読み込む。

- `analysis3_daily_plan.csv`：各日について，システムが提示した当日プラン情報  
  （planned task の一覧，提示した推奨作業時間 todayMinutes など）
- `analysis3_daily_actual.csv`：各日について，実際に記録された作業実績  
  （実行されたタスク，投入された作業時間 minutes など）

以降の処理では，日付をキーとして plan と actual を対応付け，
(1) planned / unplanned の実行率の比較，
(2) planned task に対するプラン遵守度（提示時間に対する実績時間比）の算出
を行う。


In [1]:
import pandas as pd
import numpy as np

plan = pd.read_csv("analysis3_daily_plan.csv")
actual = pd.read_csv("analysis3_daily_actual.csv")

print("plan shape:", plan.shape)
print("actual shape:", actual.shape)

plan.head(), actual.head()

plan shape: (64, 16)
actual shape: (61, 4)


(                         userId        date                todoId  \
 0  AJEH3V05QuRj9WkQva2LM8EO3vt2  2025-12-25  coQ7j50RsA9aYNNSHzXI   
 1  DgApBqtxpeVE5G5MnIVw9ThpyN93  2025-12-16  OBRscVWDrP6T2dgGwzRM   
 2  DgApBqtxpeVE5G5MnIVw9ThpyN93  2025-12-16  yFII5G1f8Yf47CTrPlWC   
 3  DgApBqtxpeVE5G5MnIVw9ThpyN93  2025-12-17  OBRscVWDrP6T2dgGwzRM   
 4  DgApBqtxpeVE5G5MnIVw9ThpyN93  2025-12-17  yFII5G1f8Yf47CTrPlWC   
 
    plannedMinutes  capMinutes  totalPlannedMinutes   spi     eacDate  \
 0              60         120                   60  3.00  2025-12-27   
 1             135         300                  300  0.00         NaN   
 2             165         300                  300  0.21  2026-07-11   
 3             135         300                  300  0.00         NaN   
 4             165         300                  300  0.21  2026-07-11   
 
   riskLevel  idealProgress  actualProgress                  deadline  \
 0        ok           0.17            0.33  2025-12-31T09:00:00.

### 日付形式の統一とデータ整合性の確認

本分析では，日付単位で「当日プラン」と「実績ログ」を対応付けて比較を行う。
そのため，両データに含まれる日付情報を
`datetime` 型に統一し，時間的な整合性を確保する。

また，分析対象となる CSV は，
「1ユーザー・1日・1タスク」に対して
1 行のみが存在することを前提としている。
この前提が崩れると，
実行率やプラン遵守度の算出に影響を及ぼすため，
以下では重複行の有無を事前に確認する。

重複の判定は，
`userId`，`date`，`todoId` をキーとして行い，
同一キーを持つ行が複数存在しないかを検証する。


In [2]:
# date を datetime に変換
plan["date"] = pd.to_datetime(plan["date"])
actual["date"] = pd.to_datetime(actual["date"])

# 重複チェック
plan_dups = plan.duplicated(subset=["userId", "date", "todoId"]).sum()
actual_dups = actual.duplicated(subset=["userId", "date", "todoId"]).sum()

print("plan duplicated keys:", plan_dups)
print("actual duplicated keys:", actual_dups)

plan duplicated keys: 0
actual duplicated keys: 0


### 当日プランと実績ログの対応付け（結合処理）

本分析では，「今日のプラン」に含まれるタスクが
実際に実行されたかどうかを評価するため，
当日プランデータと実績ログを日付単位で対応付ける。

ここでは，
当日プランに含まれるタスクを分析の基準とするため，
`analysis3_daily_plan.csv` を基準とした left join を行う。
これにより，

- 当日プランに含まれており，実績が存在するタスク
- 当日プランに含まれているが，実績が存在しないタスク

を明確に区別できる。

結合キーには，
`userId`，`date`，`todoId` を用い，
同一ユーザー・同一日・同一タスクにおける
計画と実績を1対1で対応付ける。


In [3]:
# plan と actual を left join（plan 基準）
merged = plan.merge(
    actual,
    on=["userId", "date", "todoId"],
    how="left"
)

print("merged shape:", merged.shape)
merged.head()

merged shape: (64, 17)


,userId,date,todoId,plannedMinutes,capMinutes,totalPlannedMinutes,spi,eacDate,riskLevel,idealProgress,actualProgress,deadline,deadlineDate,statsUpdatedAt,statsUpdatedDate,planUpdatedAt,actualMinutes
0,AJEH3V05QuRj9WkQva2LM8EO3vt2,2025-12-25,coQ7j50RsA9aYNNSHzXI,60,120,60,3.00,2025-12-27,ok,0.17,0.33,2025-12-31T09:00:00.000Z,2025-12-31,2025-12-25T13:22:07.428Z,2025-12-25,2025-12-25T13:21:19.998Z,30.0
1,DgApBqtxpeVE5G5MnIVw9ThpyN93,2025-12-16,OBRscVWDrP6T2dgGwzRM,135,300,300,0.00,NaN,late,1.00,0.38,2025-12-09T09:00:00.000Z,2025-12-09,2025-12-16T05:24:42.160Z,2025-12-16,2025-12-16T05:24:38.196Z,NaN
2,DgApBqtxpeVE5G5MnIVw9ThpyN93,2025-12-16,yFII5G1f8Yf47CTrPlWC,165,300,300,0.21,2026-07-11,late,0.63,0.39,2026-01-31T09:00:00.000Z,2026-01-31,2025-12-26T02:36:54.363Z,2025-12-26,2025-12-16T05:24:38.196Z,NaN
3,DgApBqtxpeVE5G5MnIVw9ThpyN93,2025-12-17,OBRscVWDrP6T2dgGwzRM,135,300,300,0.00,NaN,late,1.00,0.38,2025-12-09T09:00:00.000Z,2025-12-09,2025-12-16T05:24:42.160Z,2025-12-16,2025-12-17T09:11:35.317Z,NaN
4,DgApBqtxpeVE5G5MnIVw9ThpyN93,2025-12-17,yFII5G1f8Yf47CTrPlWC,165,300,300,0.21,2026-07-11,late,0.63,0.39,2026-01-31T09:00:00.000Z,2026-01-31,2025-12-26T02:36:54.363Z,2025-12-26,2025-12-17T09:11:35.317Z,NaN


### 日次集計（planned task の実行率とプラン遵守度）

結合後データは「1ユーザー・1日・1タスク」粒度であるため，
分析3の評価指標を算出するには日次単位への集計が必要となる。
そこで，ユーザーIDと日付でグルーピングし，
当日の planned task に関する実行状況と投入時間を集計する。

まず，計画・実績に関する列（plannedMinutes, actualMinutes など）を数値型に統一し，
欠損値は「実行されなかった（0分）」として扱えるよう
`actualMinutes` の欠損を 0 に補完した列（actualMinutes_filled）を作成する。

日次集計では，主に以下を算出する。

- **planned_total**：当日プランで提示された作業時間の合計（planned task の推奨時間合計）
- **actual_planned_total**：planned task に対して実際に投入された作業時間の合計
- **planned_tasks**：当日プランに含まれるタスク数
- **executed_planned_tasks**：planned task のうち実績が記録されたタスク数（記録の有無）
- **executed_planned_tasks_pos**：planned task のうち実績時間が正（>0）のタスク数（実行の有無）

最後に，planned_total が 0 の日を除外するための条件分岐を行い，
次式により **プラン遵守度** を定義する。

\[
\mathrm{Adherence}(d)=\frac{\sum \mathrm{ActualPlannedMinutes}(d)}{\sum \mathrm{PlannedMinutes}(d)}
\]

ここで，Adherence(d) が 1 に近いほど，
当日の計画に沿った時間配分が達成されたことを示す。


In [4]:
import numpy as np
import pandas as pd

tmp = merged.copy()

# まず planned/actual は数値化
for c in ["plannedMinutes", "capMinutes", "totalPlannedMinutes", "spi", "actualMinutes"]:
    if c in tmp.columns:
        tmp[c] = pd.to_numeric(tmp[c], errors="coerce")

tmp["actualMinutes_filled"] = tmp["actualMinutes"].fillna(0)

daily = (
    tmp.groupby(["userId", "date"], as_index=False)
    .agg(
        planned_total=("plannedMinutes", "sum"),
        actual_planned_total=("actualMinutes_filled", "sum"),
        planned_tasks=("todoId", "count"),
        executed_planned_tasks=("actualMinutes", lambda s: s.notna().sum()),
        executed_planned_tasks_pos=("actualMinutes_filled", lambda s: (s > 0).sum()),
        totalPlannedMinutes=("totalPlannedMinutes", "max"),
        capMinutes=("capMinutes", "max"),
        spi=("spi", "max"),
        # riskLevel は max だと文字の大小になるので「最頻値」にしておく（無難）
        riskLevel=("riskLevel", lambda s: s.dropna().mode().iloc[0] if s.dropna().size else np.nan),
    )
)

daily["plan_adherence"] = np.where(
    daily["planned_total"] > 0,
    daily["actual_planned_total"] / daily["planned_total"],
    np.nan
)

daily.head(), daily.shape


(                         userId       date  planned_total  \
 0  AJEH3V05QuRj9WkQva2LM8EO3vt2 2025-12-25             60   
 1  DgApBqtxpeVE5G5MnIVw9ThpyN93 2025-12-16            300   
 2  DgApBqtxpeVE5G5MnIVw9ThpyN93 2025-12-17            300   
 3  DgApBqtxpeVE5G5MnIVw9ThpyN93 2025-12-18             77   
 4  DgApBqtxpeVE5G5MnIVw9ThpyN93 2025-12-19             81   
 
    actual_planned_total  planned_tasks  executed_planned_tasks  \
 0                  30.0              1                       1   
 1                   0.0              2                       0   
 2                   0.0              2                       0   
 3                  30.0              1                       1   
 4                  50.0              2                       2   
 
    executed_planned_tasks_pos  totalPlannedMinutes  capMinutes   spi  \
 0                           1                   60         120  3.00   
 1                           0                  300         300  0.21   
 2 

### planned / unplanned task における実行率の比較

本節では，「今日のプラン」に含まれるタスク（planned task）と，
含まれないタスク（unplanned task）とで，
当日の実行率に差があるかを検証する。

まず，`plannedMinutes` が存在するタスクを planned task，
存在しないタスクを unplanned task と定義する。
次に，実績作業時間が 0 分を超えているかどうかを基準として，
タスクが実行されたか否かを二値化する。

この定義に基づき，
planned / unplanned の別に，
当日に実行されたタスクの割合（実行率）を算出する。
実行率が planned task で高くなる場合，
プラン提示が行動選択に影響を与えている可能性が示唆される。


In [5]:
# Planned / Unplanned フラグ
merged["is_planned"] = merged["plannedMinutes"].notna()

# 実行フラグ
merged["executed"] = merged["actualMinutes"].fillna(0) > 0

merged.groupby("is_planned")["executed"].mean()


,executed
is_planned,
True,0.34375


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### planned / executed フラグの定義

以降の分析では，
タスクが「当日プランに含まれていたか」，
および「実際に実行されたか」を
明確に区別して扱う必要がある。

そこで，各タスクについて以下の二つの二値フラグを定義する。

- **is_planned**：当日プランに含まれていたタスク  
  （`plannedMinutes > 0` を満たすもの）
- **is_executed**：当日に実行されたタスク  
  （`actualMinutes > 0` を満たすもの）

これにより，
planned / unplanned と executed / not executed を
組み合わせた 4 状態を区別して扱うことが可能となる。
この定義に基づき，
次節では各状態の出現頻度や割合を比較する。


In [6]:
df = merged.copy()

df["is_planned"] = df["plannedMinutes"].fillna(0) > 0
df["is_executed"] = df["actualMinutes"].fillna(0) > 0

### unplanned task の実行内容の確認

前節までの集計結果から，
unplanned task であっても一定数が当日に実行されていることが確認された。
そこで本節では，
unplanned task のうち実際に実行された事例を抽出し，
その内容を具体的に確認する。

ここでは，
「当日プランには含まれていないが，
実績作業時間が記録されたタスク」
を unplanned executed task と定義し，
日付，タスクID，実績作業時間を確認する。

この確認は，
集計結果が異常値やデータ不整合によるものではなく，
利用者の実際の行動選択を反映していることを
定性的に裏付ける目的で行う。


In [7]:
unplanned_executed = df[
    (~df["is_planned"]) & (df["is_executed"])
]

unplanned_executed[["date", "todoId", "actualMinutes"]].head()

,date,todoId,actualMinutes


### unplanned task の日次実行量の算出

前節では，
unplanned task の具体例を確認することで，
当日プランに含まれない行動が
実際に発生していることを定性的に確認した。

本節では，
これらの unplanned task が
日次単位でどの程度発生しているかを定量的に把握するため，
ユーザーIDと日付で集計を行う。

具体的には，
当日に実行された unplanned task について，

- **unplanned_tasks**：実行された unplanned task の種類数  
- **unplanned_minutes**：unplanned task に投入された作業時間の合計  

を算出する。

この集計により，
当日プラン外の行動が，
単発的な例外なのか，
あるいは一定の時間配分を占める行動なのかを
日次レベルで評価することが可能となる。


In [8]:
daily_unplanned = (
    unplanned_executed
    .groupby(["userId", "date"])
    .agg(
        unplanned_tasks=("todoId", "nunique"),
        unplanned_minutes=("actualMinutes", "sum")
    )
    .reset_index()
)

daily_unplanned.head()


,userId,date,unplanned_tasks,unplanned_minutes


### unplanned 行動の比率指標の算出

本節では，
当日に実行されたタスク全体のうち，
どの程度が unplanned task で占められているかを
日次単位で定量化する。

まず，
各ユーザー・各日について，

- **executed_tasks**：当日に実行されたタスク数  
- **unplanned_executed_tasks**：そのうち，当日プランに含まれていなかったタスク数  

を集計する。

次に，
以下の式により **unplanned 比率** を定義する。

\[
\mathrm{UnplannedRatio}(d)
= \frac{\mathrm{UnplannedExecutedTasks}(d)}
       {\mathrm{ExecutedTasks}(d)}
\]

この指標は，
当日の行動がどの程度プランから逸脱していたかを表すものであり，
値が大きいほど，
計画外の行動が多かったことを示す。

以降の分析では，
この unplanned 比率と，
プラン遵守度（plan\_adherence）や SPI，riskLevel との関係を調べることで，
プラン提示が行動の構造にどのような影響を与えているかを検討する。


In [9]:
daily_summary = (
    df.groupby(["userId", "date"])
    .agg(
        executed_tasks=("is_executed", "sum"),
        unplanned_executed_tasks=(
            "is_executed",
            lambda x: ((x) & (~df.loc[x.index, "is_planned"])).sum()
        )
    )
    .reset_index()
)

daily_summary["unplanned_ratio"] = (
    daily_summary["unplanned_executed_tasks"]
    / daily_summary["executed_tasks"]
)

daily_summary.head()


,userId,date,executed_tasks,unplanned_executed_tasks,unplanned_ratio
0,AJEH3V05QuRj9WkQva2LM8EO3vt2,2025-12-25,1,0,0.0
1,DgApBqtxpeVE5G5MnIVw9ThpyN93,2025-12-16,0,0,NaN
2,DgApBqtxpeVE5G5MnIVw9ThpyN93,2025-12-17,0,0,NaN
3,DgApBqtxpeVE5G5MnIVw9ThpyN93,2025-12-18,1,0,0.0
4,DgApBqtxpeVE5G5MnIVw9ThpyN93,2025-12-19,2,0,0.0


### planned だが未実行となったタスクの確認（未実行分析）

ここまでの分析では，
planned / unplanned の実行率や unplanned 比率を用いて，
プラン提示が行動選択に与える影響を定量化した。

一方で，プランが提示されても実行されなかったタスクが存在する場合，
その内容を確認することは，
「プランが機能しなかった要因」や
「実行困難な提示が含まれていないか」を検討するうえで重要である。

そこで本節では，
当日プランに含まれていたにもかかわらず，
当日の実績作業時間が 0 分であったタスクを抽出し，
日付，タスクID，提示作業時間（plannedMinutes）を確認する。

この確認は，
単なる集計値だけでは把握できない
未実行パターンの特徴（提示時間の大きさ，タスクの偏り等）を把握し，
後続の考察に結びつける目的で行う。


In [10]:
planned_not_executed = df[
    (df["is_planned"]) & (~df["is_executed"])
]

planned_not_executed[["date", "todoId", "plannedMinutes"]].head()

,date,todoId,plannedMinutes
1,2025-12-16,OBRscVWDrP6T2dgGwzRM,135
2,2025-12-16,yFII5G1f8Yf47CTrPlWC,165
3,2025-12-17,OBRscVWDrP6T2dgGwzRM,135
4,2025-12-17,yFII5G1f8Yf47CTrPlWC,165
8,2025-12-20,yFII5G1f8Yf47CTrPlWC,79


### planned task に対する日次評価指標（実行率・時間遵守率）

本節では，今日のプラン（planned task）に対して，
利用者がどの程度実行できたかを
日次単位で評価するための指標を定義する。

ここでは，planned task のみに対象を限定し，
ユーザーIDと日付で集計を行う。
集計により，当日プランの「件数ベース」と「時間ベース」の2種類の達成度を算出する。

算出する指標は以下の通りである。

- **planned_tasks**：当日プランに含まれるタスク数（重複除去）
- **executed_planned_tasks**：planned task のうち実行されたタスク数
- **planned_minutes**：当日プランで提示された作業時間の合計
- **executed_minutes**：planned task に実際に投入された作業時間の合計

これらから，
次の 2 指標を導入する。

1. **タスク実行率（Task Execution Rate）**
\[
\mathrm{TaskExecRate}(d)=\frac{\mathrm{ExecutedPlannedTasks}(d)}{\mathrm{PlannedTasks}(d)}
\]

2. **時間遵守率（Time Adherence Rate）**
\[
\mathrm{TimeAdhRate}(d)=\frac{\mathrm{ExecutedMinutes}(d)}{\mathrm{PlannedMinutes}(d)}
\]

タスク実行率は「プランの項目がどれだけ実行されたか」を表し，
時間遵守率は「提示された作業時間配分がどの程度達成されたか」を表す。
以降では，これらの指標と SPI / riskLevel 等との関係を分析する。


In [11]:
summary = (
    df[df["is_planned"]]
    .groupby(["userId", "date"])
    .agg(
        planned_tasks=("todoId", "nunique"),
        executed_planned_tasks=("is_executed", "sum"),
        planned_minutes=("plannedMinutes", "sum"),
        executed_minutes=("actualMinutes", "sum"),
    )
    .reset_index()
)

summary["task_execution_rate"] = (
    summary["executed_planned_tasks"] / summary["planned_tasks"]
)

summary["time_adherence_rate"] = (
    summary["executed_minutes"] / summary["planned_minutes"]
)

summary.head()

,userId,date,planned_tasks,executed_planned_tasks,planned_minutes,executed_minutes,task_execution_rate,time_adherence_rate
0,AJEH3V05QuRj9WkQva2LM8EO3vt2,2025-12-25,1,1,60,30.0,1.0,0.500000
1,DgApBqtxpeVE5G5MnIVw9ThpyN93,2025-12-16,2,0,300,0.0,0.0,0.000000
2,DgApBqtxpeVE5G5MnIVw9ThpyN93,2025-12-17,2,0,300,0.0,0.0,0.000000
3,DgApBqtxpeVE5G5MnIVw9ThpyN93,2025-12-18,1,1,77,30.0,1.0,0.389610
4,DgApBqtxpeVE5G5MnIVw9ThpyN93,2025-12-19,2,2,81,50.0,1.0,0.617284
